In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/n-beats-bax/df_bax_cleaned_till_outliers.csv


In [4]:
# It is recommended to run this code in a Kaggle notebook or a similar environment.
# First, let's install the necessary library, Darts, for time series analysis.
!pip install darts

# Now, we import all the required libraries
import pandas as pd
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.metrics import rmse
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('/kaggle/input/n-beats-bax/df_bax_cleaned_till_outliers.csv')

# --- 1. Data Preprocessing ---

# Rename the 'Vol.' column to 'Volume'
df.rename(columns={'Vol.': 'Volume'}, inplace=True)

# Drop the columns we don't need for univariate price prediction
df = df.drop(columns=['Open', 'High', 'Low', 'Volume', 'Change %'])

# Convert the 'Date' column to datetime objects and set it as the index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# --- FIX for the ValueError ---
# Resample the data to a daily frequency to make the time series regular.
# This will create rows for missing dates (like weekends and holidays).
df = df.asfreq('D')

# Forward-fill the missing 'Price' values.
# This will fill the price for weekends/holidays with the last known price.
df['Price'].fillna(method='ffill', inplace=True)
# --- End of FIX ---

# Create a Darts TimeSeries object from the 'Price' column
series = TimeSeries.from_series(df['Price']) # Darts will now infer the daily frequency

# --- 2. Train-Test Split ---

# We'll use the first 80% of the data for training and the remaining 20% for validation.
train, val = series.split_before(0.8)

# --- 3. N-BEATS Model Training ---

model = NBEATSModel(input_chunk_length=30, output_chunk_length=7, n_epochs=100, random_state=42)

# Train the model on the training data
model.fit(train, verbose=True)

# --- 4. Prediction ---

# Make predictions on the validation set
pred = model.predict(n=len(val))

# --- 5. Evaluation and Visualization ---

# Calculate the Root Mean Squared Error (RMSE)
rmse_value = rmse(val, pred)
print(f"RMSE: {rmse_value}")

# Plot the results
plt.figure(figsize=(12, 6))
series.plot(label='Actual Price')
pred.plot(label='Predicted Price')
plt.title('Price Prediction with N-BEATS')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.savefig('price_prediction_nbeats.png')

# The plot will be saved as 'price_prediction_nbeats.png'

/tmp/ipykernel_36/3263198951.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Price'].fillna(method='ffill', inplace=True)
/tmp/ipykernel_36/3263198951.py:34: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Price'].fillna(method='ffill', inplace=True)


Training: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 415.21435610939733
